In [1]:
import urllib
import json
import pandas as pd

In [2]:
account_step2_df = pd.read_csv('account_step2_df.csv')
account_step2_df[:1]

,corp_name,corp_code,bsns_year,reprt_code,data
0,신한은행,149293,2020,11012,"{""status"":""013"",""message"":""조회된 데이타가 없습니다.""}"


In [8]:
# Step3에서 DataFrame으로 만들 데이터 열 정의
account_df = pd.DataFrame(columns=["corp_name","bsns_year","reprt_code",
                                   "자산총계","부채총계","자본총계","매출액",
                                   "영업이익","당기순이익"])

for index, row in account_step2_df.iterrows(): # Step2에서 저장한 JSON 데이터를 하나씩 분석함
    
    data = json.loads(row["data"]) # data 열에 저장한 json 데이터를 python dict형식으로 읽어들임
    
    if data['status'] != '000':    # 데이터를 불러오지 못한 항목은
        continue                   # 건너뜀
    
    new_row = dict()               # json 데이터를 분석해서 account_df 에 저장할 때 사용할 새 데이터 행

    for column in ["corp_name", "bsns_year", "reprt_code"]:
        new_row[column] = row[column]    # account_step2_df 에서 그대로 복사할 열
    
    # json 데이터 중 list item에서 아래 이름들에 대해 열로 추출하는 코드
    for column in ["자산총계","부채총계","자본총계","매출액","영업이익","당기순이익"]:
        
        for item_in_list in data['list']:
            
            # OFS는 연결아닌 일반 재무제표 계정을 뜻함
            if item_in_list["account_nm"] == column and item_in_list["fs_div"] == "OFS":
                
                try:
                    # comma를 삭제하고 thstrm_amount 키의 문자열 값을 강제로 정수(int)로 변환함
                    new_row[column] = int(item_in_list["thstrm_amount"].replace(",", ""))
                except: # 변환 과정에 문제(Exception)가 발생하면
                    # int()로 변환할 수 없었던 값을 출력함
                    fmt = 'int로 분석할 수 없는 값: {}, {}, {}'
                    print(fmt.format(new_row["corp_name"], column, item_in_list["thstrm_amount"]))
                    
                    # int()로 변환할 수 없다면 무조건 0으로 처리함
                    new_row[column] = 0 
    
    # 만들어진 새 데이터를 추가하고, 새로 만들어진 df를 account_df에 할당
    account_df = account_df.append(new_row, ignore_index=True)

account_df.to_csv('account_step3_df.csv', index=False) # 정리된 데이터를 저장함

# 자산 총계를 기준으로 상위 100개의 내용을 추출함
top_100_df = account_df.sort_values(by=['자산총계'], ascending=False)[:100]
top_100_df.to_csv('account_top100.csv', index=False) # 추출한 데이터를 저장함
top_100_df

int로 분석할 수 없는 값: 예스코홀딩스, 매출액, -
int로 분석할 수 없는 값: 엔씨소프트, 매출액, -
int로 분석할 수 없는 값: 아리온, 매출액, -
int로 분석할 수 없는 값: 아리온, 영업이익, -
int로 분석할 수 없는 값: 아리온, 당기순이익, -
int로 분석할 수 없는 값: 삼영이엔씨, 당기순이익, -
int로 분석할 수 없는 값: 엠에스오토텍, 자산총계, -
int로 분석할 수 없는 값: 포티스, 매출액, -
int로 분석할 수 없는 값: 카이노스메드, 매출액, -
int로 분석할 수 없는 값: 삼성스팩2호, 매출액, -
int로 분석할 수 없는 값: 하나머스트제6호스팩, 매출액, -
int로 분석할 수 없는 값: 삼성머스트스팩3호, 매출액, -
int로 분석할 수 없는 값: 하나금융15호스팩, 매출액, -


,corp_name,bsns_year,reprt_code,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
239,삼성전자,2020,11012,218197402000000,39446957000000,178750445000000,37904100000000,4647974000000,3522241000000
463,한국전력공사,2020,11012,111049348000000,59665959000000,51383389000000,12949805000000,511584000000,264974000000
218,현대자동차,2020,11012,74466283000000,20694555000000,53771728000000,12131110000000,651424000000,482965000000
28,SK하이닉스,2020,11012,61238070000000,14000590000000,47237480000000,8235501000000,1758937000000,1203627000000
222,포스코,2020,11012,57620108949580,12217346982313,45402761967267,5884823631004,-108474793466,6645986027
...,...,...,...,...,...,...,...,...,...
238,대한해운,2020,11012,2910237160082,2231796814831,678440345251,140577520957,25880990790,-43412766018
188,삼천리,2020,11012,2867682006043,1558579695777,1309102310266,494188983866,3972721860,6147032581
314,호텔신라,2020,11012,2833970659329,1973176392183,860794267146,493893036644,-51506845371,NaN
144,쌍용양회공업,2020,11012,2814379054591,1052786614221,1761592440370,255750190341,69507942876,42273109241
